In [1]:
import requests
import pandas
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from supabase.client import Client, create_client
import datetime

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
papersDF = pandas.read_csv('csv-microRNAAN-set (1).csv')

In [3]:
for index, row in papersDF.iterrows():
    doi = row['DOI']
    pass

In [4]:
test_doi = "10.1038/s41584-020-0426-0"

In [20]:
req = requests.get(f"https://api.unpaywall.org/v2/{test_doi}?email=ryandu9221@gmail.com")
url = req.json().get('best_oa_location').get('url')
fileloader = OnlinePDFLoader(url)
loader = UnstructuredPDFLoader(fileloader.file_path)
splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
chunks = loader.load_and_split(text_splitter=splitter)
for chunk in chunks:
    chunk.metadata["doi"] = test_doi
    chunk.metadata["add_date"] = str(datetime.datetime.now().date())
    chunk.metadata.pop("source")

In [21]:
chunks[0].metadata

{'doi': '10.1038/s41584-020-0426-0', 'add_date': '2023-08-10'}

In [24]:
embedding = OpenAIEmbeddings()
supabase: Client = create_client(supabase_url=os.environ.get("SUPABASE_URL"), supabase_key=os.environ.get("SUPABASE_SERVICE_KEY"))

In [30]:
embedded = SupabaseVectorStore.from_documents(chunks, embedding, client=supabase, table_name="knowledge")